### 마커 크기 조절하기 

지진의 강도에 따라 마커 크기를 다르게 적용할 수 있습니다. 

In [1]:
import json

from plotly.graph_objs import Scattergeo, Layout
from plotly import offline

filename = 'eq_data_1_day_m1.json'
with open(filename) as f:
    all_eq_data = json.load(f)

all_eq_dicts = all_eq_data['features']

mags = []
lons = []
lats = []

# 158개의 지진 진도 정보를 리스트에 저장합니다.
for eq_dict in all_eq_dicts:
    mag = eq_dict['properties']['mag']
    # 위치 정보를 뽑아냅니다. 
    lon = eq_dict['geometry']['coordinates'][0]  # 경도
    lat = eq_dict['geometry']['coordinates'][1]  # 위도
    mags.append(mag)
    lons.append(lon)
    lats.append(lat)

data = [{
    'type' : 'scattergeo',
    'lon' : lons,
    'lat' : lats,
    'marker' : {
        'size' : [5*mag for mag in mags],
    }
}]
my_layout = Layout(title='Global Earthquakes')

fig = {'data' : data, 'layout' : my_layout}
offline.plot(fig, filename='global_earthquakes.html')

'global_earthquakes.html'

플로틀리는 아주 다양한 방식으로 데이터를 표현할 수 있으며 이 설정은 키-값 쌍을 통해 지정할 수 있습니다. 여기서는 'marker' 키를 써서 지도에 나타날 마커 크기를 지정했습니다. 

프로그램에서는 리스트 내포(리스트 컴프리헨션)을 써서 mags 리스트의 각 값에 따라 마커 크기가 자동으로 조절되게 만들었습니다. 

### 마커 색깔 정하기

각 마커의 색깔을 다르게 해서 지진의 강도에 따라 분류하는 것도 가능합니다. 플로틀리의 colorscale을 사용하면 됩니다. 

In [3]:
import json

from plotly.graph_objs import Scattergeo, Layout
from plotly import offline

filename = 'eq_data_30_day_m1.json'
with open(filename) as f:
    all_eq_data = json.load(f)

all_eq_dicts = all_eq_data['features']

mags = []
lons = []
lats = []
hover_texts = []

# 158개의 지진 진도 정보를 리스트에 저장합니다.
for eq_dict in all_eq_dicts:
    mag = eq_dict['properties']['mag']
    # 위치 정보를 뽑아냅니다. 
    lon = eq_dict['geometry']['coordinates'][0]  # 경도
    lat = eq_dict['geometry']['coordinates'][1]  # 위도
    title = eq_dict['properties']['title']
    mags.append(mag)
    lons.append(lon)
    lats.append(lat)
    hover_texts.append(title)

data = [{
    'type' : 'scattergeo',
    'lon' : lons,
    'lat' : lats,
    'text' : hover_texts,
    'marker' : {
        'size' : [5*mag for mag in mags],
        'color' : mags,
        'colorscale' : 'Viridis',
        'reversescale' : True,
        'colorbar' : {'title' : 'Magnitude'},
    }
}]
my_layout = Layout(title='Global Earthquakes')

fig = {'data' : data, 'layout' : my_layout}
offline.plot(fig, filename='global_earthquakes.html')

'global_earthquakes.html'

'color' 세팅은 컬러스케일을 어떤 값을 적용할지 정합니다. 'colorscale' 세팅은 어떤 범위의 색깔을 사용할지 정합니다. 'Viridis'는 짙은 파랑색에서 밝은 노랑색으로 변하는 색깔입니다. 'reversescale'을 True로 지정하면 지진 강도가 약할수록 밝은 노란색에 가까운 색깔이 강할 수록 짙은 파랑색에 가까운 색깔이 사용됩니다. 
'colorbar'는 지도 오른쪽에 표시되는 컬러스케일의 모양을 정합니다. 

### 텍스트 추가

지도의 지진 마커 위에 마우스를 올렸을 때 정보가 나타나는 기능을 추가해서 지도를 그리겠습니다. 기본적으로 나타나는 경도와 위도외에 진도와 위치에 대한 설명을 추가합니다. 

이를 위해 파일에서 데이터를 더 추출해서 data 딕셔너리에 추가합니다. 

# API 사용하기

애플리케이션 프로그래밍 인터페이스(API)를 사용해 자동으로 웹사이트에 특정 정보를 요청하고, 응답으로 받은 정보를 사용해 시각화를 생성해보겠습니다. 

웹 API는 프로그램과 상호작용하기 위해 설계된 웹사이트의 일부분입니다. 이런 프로그램은 URL을 사용해 정보를 요청합니다. 이런 요청을 API 호출이라 부릅니다. 요청된 데이터는 JSON이나 CSV와 같은 사용하기 쉬운 형식으로 반환됩니다. 

### 깃과 깃허브

깃허브는 프로그래머들이 프로젝트를 함께 진행하는 사이트입니다. 우리는 깃허브에서 제공하는 API를 사용해서 파이썬 프로젝트에 관한 정보를 요청하고, 플로틀리를 써서 이들 프로젝트중 어떤 것이 가장 인기있는지 알아보는 대화형 시각화를 만들 겁니다. 

깃허브 사용자들은 프로젝트가 마음에 들면 '별점'을 줄 수 있습니다. 프로그램에서는 깃허브에서 가장 많은 별점을 받은 파이썬 프로그램에 관한 정보를 자동으로 내려받는 프로그램을 만들고, 이들 정보로 시각화도 해보겠습니다. 

### API 호출을 통해 데이터 요청하기

브라우저의 주소창에 다음 주소를 입력하면 API 호출이 어떤 형태인지 볼수 있습니다. 
https://api.github.com/search/repositories?q=language:python&sort=stars

호출은 현재 깃허브에 저장된 파이썬 프로젝트가 몇 개인지 반환하고, 가장 인기 있는 파이썬 저장소에 관한 정보도 반환합니다. 

첫 번째 부분인 https://api.github.com/ 는 요청을 깃허브에서 API 호출에 응답하는 부분으로 보냅니다. 두 번째 부분인 search/repositories는 깃허브의 저장소를 검색하겠다는 의미입니다. 물음표는 매개변수가 있다는 의미이고, q는 검색(query) 이므로 그 두의 내용을 검색하겠다는 의미입니다. language:python은 주요 언어로 파이썬을 사용하는 저장소에 관한 정보를 원하며 &sort=stars는 프로젝트를 별점 순서대로 정렬하라는 의미입니다. 

### requests 설치하기

requests 패키지를 설치하면 파이썬 프로그램에서 쉽게 웹사이트에 요청을 보내고 응답을 처리할 수 있습니다. pip 명령을 통해 requests를 설치할 수 있습니다. 

In [4]:
pip install --user requests

Note: you may need to restart the kernel to use updated packages.


### API 응답 처리

자동으로 API 호출을 보내고 결과를 분석해 깃허브에서 가장 인기 있는 파이썬 프로젝트를 찾는 프로그램을 만들겠습니다. 

In [8]:
import requests 

url = 'https://api.github.com/search/repositories?q=language:python&amp;sort=stars'
headers = {'Accept' : 'application/vnd.github.v3+json'}    # API 버전 3을 사용함
r = requests.get(url, headers=headers)     # API 호출
print(f"Status code: {r.status_code}")     # 200 상태 코드 200은 성공을 의미함함

# API 응답을 저장
response_dict = r.json()

print(response_dict.keys())

Status code: 200
dict_keys(['total_count', 'incomplete_results', 'items'])


정보를 좀 더 요약해서 출력해봅시다. 먼저 요약을 살펴보면 원하는 정보를 잘 받았는지 확인할 수 있고, 관심 있는 정보에 접근하는 첫 단계이기도 합니다. 

In [6]:
import requests 

url = 'https://api.github.com/search/repositories?q=language:python&amp;sort=stars'
headers = {'Accept' : 'application/vnd.github.v3+json'}    # API 버전 3을 사용함
r = requests.get(url, headers=headers)     # API 호출
print(f"Status code: {r.status_code}")     # 200 상태 코드 200은 성공을 의미함함

# API 응답을 변수에 저장
response_dict = r.json()
print(f"Total repositories: {response_dict['total_count']}")

# 저장소에 대한 정보
repo_dicts = response_dict['items']
print(f"Repositories returned: {len(repo_dicts)}")

# 첫 번째 저장소 데이터를 출력
repo_dict = repo_dicts[0]
print(f"\nkeys: {len(repo_dict)}")
for key in sorted(repo_dict.keys()):
    print(key)

Status code: 200
Total repositories: 14813405
Repositories returned: 30

keys: 80
allow_forking
archive_url
archived
assignees_url
blobs_url
branches_url
clone_url
collaborators_url
comments_url
commits_url
compare_url
contents_url
contributors_url
created_at
default_branch
deployments_url
description
disabled
downloads_url
events_url
fork
forks
forks_count
forks_url
full_name
git_commits_url
git_refs_url
git_tags_url
git_url
has_discussions
has_downloads
has_issues
has_pages
has_projects
has_wiki
homepage
hooks_url
html_url
id
is_template
issue_comment_url
issue_events_url
issues_url
keys_url
labels_url
language
languages_url
license
merges_url
milestones_url
mirror_url
name
node_id
notifications_url
open_issues
open_issues_count
owner
private
pulls_url
pushed_at
releases_url
score
size
ssh_url
stargazers_count
stargazers_url
statuses_url
subscribers_url
subscription_url
svn_url
tags_url
teams_url
topics
trees_url
updated_at
url
visibility
watchers
watchers_count
web_commit_signoff_re

### 플로틀리를 사용해 저장소 시각화하기

시각화는 대화형 막대 그래프 형태로 생성하겠습니다. 이 그래프에서 각 막대의 높이는 해당 프로젝트가 받은 별점 숫자이며, 막대의 라벨을 클릭하면 깃허브의 프로젝트 페이지로 갈 수 있습니다. 

In [10]:
import requests 

from plotly.graph_objs import Bar
from plotly import offline

url = 'https://api.github.com/search/repositories?q=language:python&amp;sort=stars'
headers = {'Accept' : 'application/vnd.github.v3+json'}    # API 버전 3을 사용함
r = requests.get(url, headers=headers)     # API 호출
print(f"Status code: {r.status_code}")     # 200 상태 코드 200은 성공을 의미함함

# API 응답을 변수에 저장
response_dict = r.json()
repo_dicts = response_dict['items']
repo_names, stars = [], []
for repo_dict in repo_dicts:
    repo_names.append(repo_dict['name'])    # 저장소 이름을 가져와서 리스트에 저장
    stars.append(repo_dict['stargazers_count'])

# 시각화에 사용될 데이터 리스트 생성
data = [{
    'type': 'bar',
    'x': repo_names,
    'y': stars,
}]

# 그래프 레이아웃 지정
my_layout = {
    'title' : "Most-starred Python Project on GitHub",
    'xaxis' : {'title' : 'Repository'},
    'yaxis' : {'title' : 'Stars'}
}

fig = {'data' : data, 'layout' : my_layout}
offline.plot(fig, filename='python_repos.html')

Status code: 200


'python_repos.html'

### 그래프에 클릭할 수 있는 링크 추가

플로틀리는 텍스트 요소에 HTML을 쓸 수 있으므로, 그래프에 링크를 추가하는 것도 쉽습니다. 사용자가 x축의 라벨을 클릭하면 깃허브의 프로젝트 홈페이지로 이동할 수 있게 만들어 보겠습니다. 이렇게 하려면 데이터에서 프로젝트 URL을 추출하고 x축 라벨을 만들 때 이 URL을 사용해야 합니다.

In [18]:
import requests 

from plotly.graph_objs import Bar
from plotly import offline

url = 'https://api.github.com/search/repositories?q=language:python&amp;sort=stars'
headers = {'Accept' : 'application/vnd.github.v3+json'}    # API 버전 3을 사용함
r = requests.get(url, headers=headers)     # API 호출
print(f"Status code: {r.status_code}")     # 200 상태 코드 200은 성공을 의미함함

# API 응답을 변수에 저장
response_dict = r.json()
repo_dicts = response_dict['items']
stars = [], []
repo_links = []     # 저장소들의 URL을 가져와서 저장할 리스트
labels = []
for repo_dict in repo_dicts:
    repo_names = repo_dict['name']    # 저장소 이름을 가져와서 리스트에 저장
    stars.append(repo_dict['stargazers_count'])
    repo_url = repo_dict['html_url']
    repo_link = f"<a href='{repo_url}>{repo_name}</a>"
    repo_link.append(repo_link)
    owner = repo_dict['owner']['login']
    description = repo_dict['description']
    label = f"{owner}<br />{description}"
    labels.append(label)

# 시각화에 사용될 데이터 리스트 생성
data = [{
    'type': 'bar',
    'x': repo_links,
    'y': stars,
    'hovertext' : labels,
    'marker' : {
        'color' : 'rgb(60, 100, 150)',
        'line' : {'width' : 1.5, 'color' : 'rgb(25, 25, 25)'}
    },                
    'opacity' : 0.6
}]

# 그래프 레이아웃 지정
my_layout = {
    'title' : "Most-starred Python Project on GitHub",
    'titlefont' : {'size' : 28},
    'xaxis' : {
        'title' : 'Repository',
        'titlefont' : {'size' : 24},
        'tickfont' : {'size' : 14},
    },
    'yaxis' : {
        'title' : 'Stars',
        'titlefont' : {'size' : 24},
        'tickfont' : {'size' : 14},
    }
            
}

fig = {'data' : data, 'layout' : my_layout}
offline.plot(fig, filename='python_repos.html')

Status code: 200


AttributeError: 'tuple' object has no attribute 'append'

# Pygame

파이게임을 이용하여 게임을 만들기 위해서는 라이브러리를 인스톨해야 합니다. 

In [19]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


설치된 파이게임 버전을 체크해 봅니다.

In [20]:
import pygame

print(pygame.ver)

pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
2.5.2


In [2]:
import pygame
from pygame.locals import *
from sys import exit

# 백그라운드 이미지와 물고기 이미지 위치를 문자열 형태로 저장합니다. 
ocean_image_filename = 'ocean.jpg'
fish_image_file_name = 'fish.jpg'

# 파이게임 환경을 초기화합니다. 
pygame.init()

# 화면으로 사용될 표면을 생성합니다. 
screen = pygame.display.set_mode((640, 480), 0, 32)
pygame.display.set_caption("Hello, World!")

# 배경 이미지를 로드합니다. 
background = pygame.image.load(ocean_image_filename).convert()
fish_cursor = pygame.image.load(fish_image_file_name).convert_alpha()

# 메시지 루프를 돌려가면서 프로그램을 종료시킵니다. 
while True:
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            exit()
            
    # 배경화면을 출력
    screen.blit(background, (0,0))

    # 마우스의 x, y좌표를 가져옵니다.
    x, y = pygame.mouse.get_pos()
    x -= fish_cursor.get_width() / 2
    y -= fish_cursor.get_height() / 2

    screen.blit(fish_cursor, (x, y))

    pygame.display.update()



SystemExit: 

C:\Users\admin\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## set_mode 함수의 세 번째 인자로 올수 있는 플래그

|Flag|설명|
|:-|:-|
|FULLSCREEN|전체화면을 생성합니다.|
|DOUBLEBUF|현재 서피스를 더블 버퍼 방식으로 생성합니다.|
|HWSURFACE|그래픽 메모리를 사용하는 표면을 생성합니다.|
|OPENGL|오픈지엘 라이브러리를 사용하는|
|RESIZABLE|사이즈 변경 가능한 표면을 생성합니다.|
|NOFRAME|테두리가 없는 표면을 생성합니다.|

## 키보드 이벤트 처리

키가 눌렀을때 KEYDOWN 이벤트가 발생하고 키를 놓으면 KEYUP 이벤트가 발생이 됩니다. 아래의 코드는 키로 화면의 무언가를 이동하기 위해 KEYUP 및 KEYDOWN 이벤트에 어떻게 응답할 수 있는지 보여줍니다. 코드에서는 배경이미지를 키보드의 왼쪽 또는 오른쪽 화살표를 눌렀을 때 해당 방향으로 이동시킵니다. 

In [6]:
import pygame
from pygame.locals import *
from sys import exit

# 백그라운드 이미지와 물고기 이미지 위치를 문자열 형태로 저장합니다. 
ocean_image_filename = 'ocean.jpg'

# 파이게임 환경을 초기화합니다. 
pygame.init()

# 화면으로 사용될 표면을 생성합니다. 
screen = pygame.display.set_mode((640, 480), 0, 32)
pygame.display.set_caption("Hello, World!")

# 배경 이미지를 로드합니다. 
background = pygame.image.load(ocean_image_filename).convert()

x, y = 0, 0
move_x, move_y = 0, 0

while True:
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            exit()
        # 키보드가 눌렀을 때의 이벤트 처리를 합니다.
        if event.type == KEYDOWN:
            if event.key == K_LEFT: 
                move_x = -1
            elif event.key == K_RIGHT:
                move_x = +1
            elif event.key == K_UP:
                move_y = -1
            elif event.key == K_DOWN:
                move_y = +1
        elif event.type == KEYUP:
            if event.key == K_LEFT:
                move_x = 0
            elif event.key == K_RIGHT:
                move_x = 0
            elif event.key == K_UP:
                move_y = 0
            elif event.key == K_DOWN:
                move_y = 0
    x += move_x
    y += move_y

    # 게임 화면에 배경 그림을 복사합니다. 
    screen.fill((0, 0, 0))    # 배경을 검정색으로 채웁니다.
    screen.blit(background, (x,y))

    # 실제 화면을 모니터에 출력합니다. 
    pygame.display.update()
            


SystemExit: 

### 풀스크린 모드와 창모드를 변경합니다. 

In [1]:
import pygame
from pygame.locals import *
from sys import exit

# 백그라운드 이미지와 물고기 이미지 위치를 문자열 형태로 저장합니다. 
ocean_image_filename = 'ocean.jpg'

# 파이게임 환경을 초기화합니다. 
pygame.init()

# 화면으로 사용될 표면을 생성합니다. 
screen = pygame.display.set_mode((640, 480), 0, 32)
pygame.display.set_caption("Hello, World!")

# 배경 이미지를 로드합니다. 
background = pygame.image.load(ocean_image_filename).convert()


Fullscreen = False

x, y = 0, 0
move_x, move_y = 0, 0

while True:
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            exit()
        # 키보드가 눌렀을 때의 이벤트 처리를 합니다.
        if event.type == KEYDOWN:
            if event.key == K_f:
                Fullscreen = not Fullscreen
                if Fullscreen:
                    screen = pygame.display.set_mode((640, 480), FULLSCREEN, 32)
                else:
                    screen = pygame.display.set_mode((640, 480), 0, 32)
                    
            if event.key == K_LEFT: 
                move_x = -1
            elif event.key == K_RIGHT:
                move_x = +1
            elif event.key == K_UP:
                move_y = -1
            elif event.key == K_DOWN:
                move_y = +1
        elif event.type == KEYUP:
            if event.key == K_LEFT:
                move_x = 0
            elif event.key == K_RIGHT:
                move_x = 0
            elif event.key == K_UP:
                move_y = 0
            elif event.key == K_DOWN:
                move_y = 0
    x += move_x
    y += move_y

    # 게임 화면에 배경 그림을 복사합니다. 
    screen.fill((0, 0, 0))    # 배경을 검정색으로 채웁니다.
    screen.blit(background, (x,y))

    # 실제 화면을 모니터에 출력합니다. 
    pygame.display.update()

pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\admin\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### 창 모드

창모드를 사용하여 화면을 생성한 경우 일반적으로 창 모서리를 클릭하고 마우스를 드래그하여 크기를 조절할 수 있습니다. 
파이게임에서는 set_mode를 호출할 때 RESIZABLE 플래그를 사용하여 이 작업을 수행할 수 있습니다. 파이게임은 사용자가 창의 새 너비와 높이를 포함하는 VIDEORESIZE 이벤트를 보내 창 크기 변경 여부를 알립니다. 

In [3]:
import pygame
from pygame.locals import *
from sys import exit

# 백그라운드 이미지와 물고기 이미지 위치를 문자열 형태로 저장합니다. 
ocean_image_filename = 'ocean.jpg'

# 파이게임 환경을 초기화합니다. 
pygame.init()

# 화면으로 사용될 표면을 생성합니다. 
screen = pygame.display.set_mode((640, 480), 0, 32)
pygame.display.set_caption("Hello, World!")

# 배경 이미지를 로드합니다. 
background = pygame.image.load(ocean_image_filename).convert()


Fullscreen = False
screen_width


x, y = 0, 0
move_x, move_y = 0, 0

while True:
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            exit()
        # 키보드가 눌렀을 때의 이벤트 처리를 합니다.
        if event.type == KEYDOWN:
            if event.type == VIDEORESIZE:
                SCREEN_SIZE = event.size
                screen = pygame.display.set_mode((640, 480), RESIZABLE, 32)
                
            if event.key == K_f:
                Fullscreen = not Fullscreen
                if Fullscreen:
                    screen = pygame.display.set_mode((640, 480), FULLSCREEN,32)
                else:
                    screen = pygame.display.set_mode((640, 480), RESIZABLE, 32)
                    
            if event.key == K_LEFT: 
                move_x = -1
            elif event.key == K_RIGHT:
                move_x = +1
            elif event.key == K_UP:
                move_y = -1
            elif event.key == K_DOWN:
                move_y = +1
        elif event.type == KEYUP:
            if event.key == K_LEFT:
                move_x = 0
            elif event.key == K_RIGHT:
                move_x = 0
            elif event.key == K_UP:
                move_y = 0
            elif event.key == K_DOWN:
                move_y = 0
            screen_width, screen_height = SCREEN_SIZE
    x += move_x
    y += move_y

    # 게임 화면에 배경 그림을 복사합니다. 
    screen.fill((0, 0, 0))    # 배경을 검정색으로 채웁니다.
    for y in range(0, screen_height, background.get_height()):
        for x in range(0, screen_width, background.get_width()):
            screen.blit(background, (x,y))

    # 실제 화면을 모니터에 출력합니다. 
    pygame.display.update()

NameError: name 'screen_height' is not defined

In [ ]:
background_image_filename = 'ocean.jpg'
import pygame
from pygame.locals import *
from sys import exit

SCREEN_SIZE = (640, 480)

pygame.init()
screen = pygame.display.set_mode(SCREEN_SIZE, RESIZABLE, 32)

background = pygame.image.load(background_image_filename).convert()

while True:
    event = pygame.event.wait()
    if event.type == QUIT:
        pygame.quit()
        exit()

    if event.type == VIDEORESIZE:
        SCREEN_SIZE = event.size
        screen = pygame.display.set_mode(SCREEN_SIZE, RESIZABLE, 32)
        pygame.display.set_caption("Window resized to" + str(event.size))

    screen_width, screen_height = SCREEN_SIZE
    for y in range(0, screen_height, background.get_height()):
        for x in range(0, screen_width, background.get_width()):
            screen.blit(background, (x, y))
    pygame.display.update()

pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
